In [35]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score
from scipy.stats import chi2_contingency
# from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, precision_recall_fscore_support
import warnings
import os


In [47]:
# Load the dataset
a1 = pd.read_excel("./data/case_study1.xlsx")
a2 = pd.read_excel("./data/case_study2.xlsx")


df1 = a1.copy()
df2 = a2.copy()

# print(df1.head())
df1.head(5)


,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,CC_TL,Consumer_TL,Gold_TL,Home_TL,PL_TL,Secured_TL,Unsecured_TL,Other_TL,Age_Oldest_TL,Age_Newest_TL
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0,0,1,0,4,1,4,0,72,18
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0,1,0,0,0,0,1,0,7,7
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0,6,1,0,0,2,6,0,47,2
3,4,1,0,1,1,0,1.000,0.0,1.000,0.000,...,0,0,0,0,0,0,1,1,5,5
4,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0,0,0,0,0,3,0,2,131,32


In [48]:
# Since there is very less null records in df1 i.e -99999
df1 = df1.loc[df1["Age_Oldest_TL"]!=-99999]

In [49]:
# we are removing those columns from df2 where count of null values in a column is > 10000
columns_to_be_removed = []
for cols in df2.columns:
    if(df2.loc[df2[cols]==-99999].shape[0] > 10000):
        columns_to_be_removed.append(cols)

In [50]:
len(columns_to_be_removed)

8

In [51]:
df2 = df2.drop(columns_to_be_removed , axis=1)

In [52]:
# now remove the row which is null simply in other cases

for col in df2.columns:
    df2 = df2.loc[df2[col]!=-99999]

In [53]:
df2.isna().sum()

PROSPECTID                    0
time_since_recent_payment     0
num_times_delinquent          0
max_recent_level_of_deliq     0
num_deliq_6mts                0
num_deliq_12mts               0
num_deliq_6_12mts             0
num_times_30p_dpd             0
num_times_60p_dpd             0
num_std                       0
num_std_6mts                  0
num_std_12mts                 0
num_sub                       0
num_sub_6mts                  0
num_sub_12mts                 0
num_dbt                       0
num_dbt_6mts                  0
num_dbt_12mts                 0
num_lss                       0
num_lss_6mts                  0
num_lss_12mts                 0
recent_level_of_deliq         0
tot_enq                       0
CC_enq                        0
CC_enq_L6m                    0
CC_enq_L12m                   0
PL_enq                        0
PL_enq_L6m                    0
PL_enq_L12m                   0
time_since_recent_enq         0
enq_L12m                      0
enq_L6m 

In [43]:
df1.isna().sum()

PROSPECTID              0
Total_TL                0
Tot_Closed_TL           0
Tot_Active_TL           0
Total_TL_opened_L6M     0
Tot_TL_closed_L6M       0
pct_tl_open_L6M         0
pct_tl_closed_L6M       0
pct_active_tl           0
pct_closed_tl           0
Total_TL_opened_L12M    0
Tot_TL_closed_L12M      0
pct_tl_open_L12M        0
pct_tl_closed_L12M      0
Tot_Missed_Pmnt         0
Auto_TL                 0
CC_TL                   0
Consumer_TL             0
Gold_TL                 0
Home_TL                 0
PL_TL                   0
Secured_TL              0
Unsecured_TL            0
Other_TL                0
Age_Oldest_TL           0
Age_Newest_TL           0
dtype: int64

In [54]:
# checking for the common columns in both the dataframe

for i in df1.columns:
    if i in df2.columns:
        print(i)

PROSPECTID


In [55]:
df = pd.merge(df1,df2 , how="inner" , left_on="PROSPECTID" , right_on="PROSPECTID")

In [57]:
df.head(10)

,PROSPECTID,Total_TL,Tot_Closed_TL,Tot_Active_TL,Total_TL_opened_L6M,Tot_TL_closed_L6M,pct_tl_open_L6M,pct_tl_closed_L6M,pct_active_tl,pct_closed_tl,...,pct_PL_enq_L6m_of_L12m,pct_CC_enq_L6m_of_L12m,pct_PL_enq_L6m_of_ever,pct_CC_enq_L6m_of_ever,HL_Flag,GL_Flag,last_prod_enq2,first_prod_enq2,Credit_Score,Approved_Flag
0,1,5,4,1,0,0,0.000,0.0,0.200,0.800,...,0.0,0.0,0.000,0.000,1,0,PL,PL,696,P2
1,2,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0.0,0.0,0.000,0.000,0,0,ConsumerLoan,ConsumerLoan,685,P2
2,3,8,0,8,1,0,0.125,0.0,1.000,0.000,...,0.0,0.0,0.000,0.000,1,0,ConsumerLoan,others,693,P2
3,5,3,2,1,0,0,0.000,0.0,0.333,0.667,...,0.0,0.0,0.000,0.000,0,0,AL,AL,753,P1
4,6,6,5,1,0,0,0.000,0.0,0.167,0.833,...,1.0,0.0,0.429,0.000,1,0,ConsumerLoan,PL,668,P3
5,8,6,4,2,0,0,0.000,0.0,0.333,0.667,...,0.0,0.0,0.000,0.000,0,0,ConsumerLoan,others,676,P2
6,9,1,0,1,0,0,0.000,0.0,1.000,0.000,...,0.0,0.0,0.000,0.000,0,0,ConsumerLoan,others,658,P4
7,11,7,2,5,1,0,0.143,0.0,0.714,0.286,...,0.0,1.0,0.000,0.333,0,0,CC,others,685,P2
8,13,2,2,0,0,0,0.000,0.0,0.000,1.000,...,0.0,0.0,0.000,0.000,0,0,ConsumerLoan,others,669,P2
9,14,2,1,1,1,0,0.500,0.0,0.500,0.500,...,0.0,0.0,0.000,0.000,0,0,ConsumerLoan,ConsumerLoan,687,P2


` We will divide the numerical and categorical variables separately`